In [1]:
!pip install selenium webdriver-manager beautifulsoup4 pandas


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup


In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in background
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [13]:
url = "https://www.shl.com/solutions/products/product-catalog/"
driver.get(url)
time.sleep(5)  # Wait for JS to load


In [14]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
cards = soup.find_all('div', class_='product-catalog-item')

data = []

for card in cards:
    title = card.find('div', class_='product-catalog-title')
    subtitle = card.find('div', class_='product-catalog-subtitle')
    description = card.find('div', class_='product-catalog-excerpt')

    data.append({
        "Title": title.get_text(strip=True) if title else '',
        "Subtitle": subtitle.get_text(strip=True) if subtitle else '',
        "Description": description.get_text(strip=True) if description else ''
    })


In [15]:
df = pd.DataFrame(data)
df.head()


""


In [12]:
df.to_csv('/Users/anshumangupta/Desktop/shl_assessments.csv', index=False)


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set Chrome options to run headless (optional)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # comment this if you want to see the browser
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Start driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the SHL catalog page
driver.get("https://www.shl.com/solutions/products/product-catalog/")

# Handle cookie banner
try:
    accept_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    accept_btn.click()
    print("✅ Accepted cookies")
except Exception as e:
    print("⚠️ No cookie banner found or already accepted")

# Scroll to load more cards
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
print("✅ Scrolling done")

# Parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.find_all("div", class_="product-catalog-item")
print(f"🧾 Found {len(cards)} assessments")

# Extract data from each card
data = []
for card in cards:
    title = card.find("h3").text.strip() if card.find("h3") else ""
    description = card.find("div", class_="description").text.strip() if card.find("div", class_="description") else ""
    link_tag = card.find("a", href=True)
    url = "https://www.shl.com" + link_tag["href"] if link_tag else ""
    data.append({
        "title": title,
        "description": description,
        "url": url
    })

# Save to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("shl_assessments.csv", index=False)
print("✅ Data saved to shl_assessments.csv")

# Close the browser
driver.quit()


⚠️ No cookie banner found or already accepted
✅ Scrolling done
🧾 Found 0 assessments
✅ Data saved to shl_assessments.csv


⚠️ No cookie popup found
✅ Scrolling completed
❌ Product cards not found
🔍 Found 0 cards
Empty DataFrame
Columns: []
Index: []
✅ Data saved to CSV


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--headless=new')  # Optional: run headless for speed/stability

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.shl.com/solutions/products/product-catalog/?page={}&type=2"
data = []

# Loop through first 30 pages
for page in range(1, 31):
    url = base_url.format(page)
    driver.get(url)
    print(f"🔄 Scraping page {page}...")

    # Wait for table to load
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table"))
        )
    except:
        print(f"❌ Table not found on page {page}")
        continue

    # Parse page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table")

    for table in tables:
        rows = table.find("tbody").find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 4:
                continue

            # Column 1: Assessment name + link
            name_tag = cols[0].find("a")
            if not name_tag:
                continue
            name = name_tag.text.strip()
            url = "https://www.shl.com" + name_tag["href"]

            # Column 2: Remote testing
            remote = "✅" if cols[1].find("svg") else "❌"

            # Column 3: Adaptive/IRT
            adaptive = "✅" if cols[2].find("svg") else "❌"

            # Column 4: Test Type
            test_type = cols[3].text.strip().replace("\n", " ")

            data.append({
                "name": name,
                "url": url,
                "remote_testing": remote,
                "adaptive_IRT": adaptive,
                "test_type": test_type
            })

    time.sleep(1)  # Respectful delay

driver.quit()

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("shl_prepackaged_job_solutions.csv", index=False)
print("✅ All data saved to shl_prepackaged_job_solutions.csv")


🔄 Scraping page 1...
🔄 Scraping page 2...
🔄 Scraping page 3...
🔄 Scraping page 4...
🔄 Scraping page 5...
🔄 Scraping page 6...
🔄 Scraping page 7...
🔄 Scraping page 8...
🔄 Scraping page 9...
🔄 Scraping page 10...
🔄 Scraping page 11...
🔄 Scraping page 12...
🔄 Scraping page 13...
🔄 Scraping page 14...
🔄 Scraping page 15...
🔄 Scraping page 16...


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: Unable to receive message from renderer
  (Session info: chrome=135.0.7049.41)
Stacktrace:
0   chromedriver                        0x000000010050df14 cxxbridge1$str$ptr + 2816404
1   chromedriver                        0x00000001005061cc cxxbridge1$str$ptr + 2784332
2   chromedriver                        0x000000010004e40c cxxbridge1$string$len + 93024
3   chromedriver                        0x00000001000388d4 cxxbridge1$string$len + 4136
4   chromedriver                        0x000000010003864c cxxbridge1$string$len + 3488
5   chromedriver                        0x0000000100036430 chromedriver + 189488
6   chromedriver                        0x0000000100036e8c chromedriver + 192140
7   chromedriver                        0x00000001000513bc cxxbridge1$string$len + 105232
8   chromedriver                        0x00000001000d6d60 cxxbridge1$string$len + 6

In [11]:
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('--headless=new')  # Optional
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

data = []

# Process in batches of 5 pages
for batch_start in range(1, 31, 5):
    driver = create_driver()
    
    for page in range(batch_start, min(batch_start + 5, 31)):
        url = f"https://www.shl.com/solutions/products/product-catalog/?page={page}&type=2"
        try:
            driver.get(url)
            print(f"🔄 Scraping page {page}...")

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table"))
            )

            soup = BeautifulSoup(driver.page_source, "html.parser")
            tables = soup.find_all("table")

            for table in tables:
                rows = table.find("tbody").find_all("tr")
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) < 4:
                        continue

                    name_tag = cols[0].find("a")
                    if not name_tag:
                        continue
                    name = name_tag.text.strip()
                    url = "https://www.shl.com" + name_tag["href"]
                    remote = "✅" if cols[1].find("svg") else "❌"
                    adaptive = "✅" if cols[2].find("svg") else "❌"
                    test_type = cols[3].text.strip().replace("\n", " ")

                    data.append({
                        "name": name,
                        "url": url,
                        "remote_testing": remote,
                        "adaptive_IRT": adaptive,
                        "test_type": test_type
                    })
        except Exception as e:
            print(f"❌ Error on page {page}: {e}")
        time.sleep(1)

    driver.quit()  # Quit every 5 pages

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("shl_prepackaged_job_solutions.csv", index=False)
print("✅ Done. Data saved to CSV.")


🔄 Scraping page 1...
🔄 Scraping page 2...
🔄 Scraping page 3...
🔄 Scraping page 4...
🔄 Scraping page 5...
🔄 Scraping page 6...
🔄 Scraping page 7...
🔄 Scraping page 8...
🔄 Scraping page 9...
🔄 Scraping page 10...
🔄 Scraping page 11...
🔄 Scraping page 12...
🔄 Scraping page 13...
🔄 Scraping page 14...
🔄 Scraping page 15...
🔄 Scraping page 16...
🔄 Scraping page 17...
🔄 Scraping page 18...
🔄 Scraping page 19...
🔄 Scraping page 20...
🔄 Scraping page 21...
🔄 Scraping page 22...
🔄 Scraping page 23...
🔄 Scraping page 24...
🔄 Scraping page 25...
🔄 Scraping page 26...
🔄 Scraping page 27...
🔄 Scraping page 28...
🔄 Scraping page 29...
🔄 Scraping page 30...
✅ Done. Data saved to CSV.


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument("--headless=new")  # Optional: uncomment to run headless
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

data = []

for i, start in enumerate(range(0, 373, 12)):
    # Restart driver every 5 pages to avoid crashes
    if i % 5 == 0:
        if 'driver' in locals():
            driver.quit()
        driver = create_driver()

    url = f"https://www.shl.com/solutions/products/product-catalog/?start={start}&type=1&type=1"
    print(f"🔄 Scraping page starting at {start}...")

    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")
        tables = soup.find_all("table")

        for table in tables:
            rows = table.find("tbody").find_all("tr")
            for row in rows:
                cols = row.find_all("td")
                if len(cols) < 4:
                    continue

                name_tag = cols[0].find("a")
                if not name_tag:
                    continue

                name = name_tag.text.strip()
                test_url = "https://www.shl.com" + name_tag["href"]
                remote = "✅" if cols[1].find("svg") else "❌"
                adaptive = "✅" if cols[2].find("svg") else "❌"
                test_type = cols[3].text.strip().replace("\n", " ")

                data.append({
                    "name": name,
                    "url": test_url,
                    "remote_testing": remote,
                    "adaptive_IRT": adaptive,
                    "test_type": test_type
                })
    except Exception as e:
        print(f"❌ Error on page starting at {start}: {e}")
    time.sleep(1)

# Final cleanup
if 'driver' in locals():
    driver.quit()

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("shl_prepackaged_job_solutions.csv", index=False)
print("✅ Done! All pages scraped and saved to CSV.")


🔄 Scraping page starting at 0...
🔄 Scraping page starting at 12...
🔄 Scraping page starting at 24...
🔄 Scraping page starting at 36...
🔄 Scraping page starting at 48...
🔄 Scraping page starting at 60...
🔄 Scraping page starting at 72...
🔄 Scraping page starting at 84...
🔄 Scraping page starting at 96...
🔄 Scraping page starting at 108...
❌ Error on page starting at 108: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: Unable to receive message from renderer
  (Session info: chrome=135.0.7049.41)
Stacktrace:
0   chromedriver                        0x0000000105231f14 cxxbridge1$str$ptr + 2816404
1   chromedriver                        0x000000010522a1cc cxxbridge1$str$ptr + 2784332
2   chromedriver                        0x0000000104d7240c cxxbridge1$string$len + 93024
3   chromedriver                        0x0000000104d5c8d4 cxxbridge1$string$len + 4136
4   chromedriver                        0x0000000104d5c64c cxxbridge1$string$l

In [21]:
!pip install sentence-transformers


In [23]:
# Optional: Disable TensorFlow support in transformers if not using it (avoids tf_keras issue)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# Install required packages if not already installed
# !pip install pandas beautifulsoup4 requests sentence-transformers

import pandas as pd
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from IPython.display import display, HTML

# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load SHL product catalog CSV
df = pd.read_csv("shl_prepackaged_job_solutions.csv")  # Replace with actual path if needed

# Rename columns for clarity
df = df.rename(columns={
    "name": "Assessment Name",
    "url": "URL",
    "remote_testing": "Remote Testing Support",
    "adaptive_IRT": "Adaptive/IRT Support",
    "test_type": "Test Type"
})

# Add default duration if not scraped
df["Duration"] = "Not Available"

# Convert support fields from emoji to Yes/No
df["Remote Testing Support"] = df["Remote Testing Support"].map({"✅": "Yes", "❌": "No"})
df["Adaptive/IRT Support"] = df["Adaptive/IRT Support"].map({"✅": "Yes", "❌": "No"})

# Encode all product names into embeddings
product_embeddings = model.encode(df["Assessment Name"].tolist(), convert_to_tensor=True)

# (Optional) Function to fetch webpage text if input is a URL
def fetch_text_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        return text[:3000]
    except Exception as e:
        return f"Error fetching URL: {str(e)}"

# Main function to recommend top 10 assessments
def recommend_assessments(input_text):
    query_embedding = model.encode(input_text, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, product_embeddings)[0]
    top_results = similarities.argsort(descending=True)[:10]
    return df.iloc[top_results.cpu().numpy()].reset_index(drop=True)

# INPUT: text or URL
input_mode = "text"  # Set to "url" if input is a webpage
input_data = "Looking to hire mid-level professionals who are proficient in Python, SQL and JavaScript. Need an assessment package that can test all skills with max duration of 60 minutes."

if input_mode == "url":
    input_data = fetch_text_from_url(input_data)

# Get recommendations
results = recommend_assessments(input_data)

# Make URL clickable in output
def make_clickable(val):
    return f'<a href="{val}" target="_blank">{val}</a>'

results_html = results.copy()
results_html["URL"] = results_html["URL"].apply(make_clickable)

# Display results as HTML table
display(HTML(results_html.to_html(escape=False, index=False)))


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.